In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as pl
import os
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization, Activation, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, LeakyReLU, concatenate
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow import clip_by_value
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
import pickle
from random import sample
import gc

In [ ]:
train_images = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Images\\train_images.parquet').read().to_pandas()
test_images = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Images\\test_images.parquet').read().to_pandas()
val_images = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Images\\val_images.parquet').read().to_pandas()

train_labels = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Labels\\train_labels.parquet').read().to_pandas()
test_labels = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Labels\\test_labels.parquet').read().to_pandas()
val_labels = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Labels\\val_labels.parquet').read().to_pandas()

train_images = np.array(train_images).reshape(train_images.shape[0],48,48,1)
test_images = np.array(test_images).reshape(test_images.shape[0],48,48,1)
val_images = np.array(val_images).reshape(val_images.shape[0],48,48,1)

In [ ]:
train_images = np.concatenate([train_images, test_images])
train_labels = np.concatenate([train_labels, test_labels])

In [ ]:
cnn_input = Input(shape=(48,48,1))
x = Conv2D(224, kernel_size=6, activation=LeakyReLU(alpha=0.021))(cnn_input)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.5)(x)
x = Conv2D(448, kernel_size=2, activation=LeakyReLU(alpha=0.021))(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
cnn_output = Dense(7, activation='softmax')(x)

In [ ]:
cnn = Model(inputs=[cnn_input], outputs=cnn_output)
optimizer = optimizers.Adam(learning_rate=0.000125)
cnn.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
cnn_history = cnn.fit(train_images, train_labels, validation_data=(val_images, val_labels),
                      batch_size=200, epochs=600)

In [ ]:
cnn.save('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Models\\Convolutional_Fully Trained\\Model.h5')
with open('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Models\\Convolutional_Fully Trained\\Training History.pkl','wb') as outfile:
    pickle.dump(cnn_history.history, outfile)

In [ ]:
cnn_predicted_categories = np.argmax(cnn.predict(val_images), axis=1)
val_labels = np.argmax(np.array(val_labels), axis=1)

In [ ]:
cnn_cm = confusion_matrix(cnn_predicted_categories, val_labels, labels=[0,1,2,3,4,5,6])
cnn_cm

In [ ]:
correct_classifications = 0
individual_classification_dict = {0:[0,0],
                                  1:[0,0],
                                  2:[0,0],
                                  3:[0,0],
                                  4:[0,0],
                                  5:[0,0],
                                  6:[0,0]}
for pred in range(cnn_predicted_categories.shape[0]):        
    # If the correct classification is Angry, tally the observation and determine whether the model was correct
    if val_labels[pred] == 0:
        individual_classification_dict[val_labels[pred]][0] += 1
        if cnn_predicted_categories[pred] == val_labels[pred]:
            individual_classification_dict[val_labels[pred]][1] += 1
            
    # If the correct classification is Disgust, tally the observation and determine whether the model was correct        
    elif val_labels[pred] == 1:
        individual_classification_dict[val_labels[pred]][0] += 1
        if cnn_predicted_categories[pred] == val_labels[pred]:
            individual_classification_dict[val_labels[pred]][1] += 1
            
    # If the correct classification is Fear, tally the observation and determine whether the model was correct        
    elif val_labels[pred] == 2:
        individual_classification_dict[val_labels[pred]][0] += 1
        if cnn_predicted_categories[pred] == val_labels[pred]:
            individual_classification_dict[val_labels[pred]][1] += 1
            
    # If the correct classification is Happy, tally the observation and determine whether the model was correct        
    elif val_labels[pred] == 3:
        individual_classification_dict[val_labels[pred]][0] += 1
        if cnn_predicted_categories[pred] == val_labels[pred]:
            individual_classification_dict[val_labels[pred]][1] += 1
            
    # If the correct classification is Sad, tally the observation and determine whether the model was correct        
    elif val_labels[pred] == 4:
        individual_classification_dict[val_labels[pred]][0] += 1
        if cnn_predicted_categories[pred] == val_labels[pred]:
            individual_classification_dict[val_labels[pred]][1] += 1
            
    # If the correct classification is Surprise, tally the observation and determine whether the model was correct        
    elif val_labels[pred] == 5:
        individual_classification_dict[val_labels[pred]][0] += 1
        if cnn_predicted_categories[pred] == val_labels[pred]:
            individual_classification_dict[val_labels[pred]][1] += 1
            
    # If the correct classification is Neutral, tally the observation and determine whether the model was correct        
    elif val_labels[pred] == 6:
        individual_classification_dict[val_labels[pred]][0] += 1
        if cnn_predicted_categories[pred] == val_labels[pred]:
            individual_classification_dict[val_labels[pred]][1] += 1

In [ ]:
emotion_dict = {0:'Angry',
                1:'Disgust',
                2:'Fear',
                3:'Happy',
                4:'Sad',
                5:'Surprise',
                6:'Neutral'}

In [ ]:
for i in range(7):
    emotion_accuracy = individual_classification_dict[i][1]/individual_classification_dict[i][0]
    emotion = emotion_dict[i]
    print(f'{emotion} accuracy: {round(100*emotion_accuracy,1)}% ({individual_classification_dict[i][0]} images)')

In [ ]:
cnn_accuracy = (cnn_cm[0,0]+cnn_cm[1,1]+cnn_cm[2,2]+cnn_cm[3,3]+cnn_cm[4,4]+cnn_cm[5,5]+cnn_cm[6,6])/np.sum(cnn_cm)
print(f'Conv-Net Accuracy: {round(100*cnn_accuracy,1)}%')